In [ ]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
default_bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name

In [ ]:
code_tarname = 'acc_hku_vec_model'

!rm -rf {code_tarname}.tar.gz
!rm -rf {code_tarname}/.ipynb_checkpoints
!tar czvf {code_tarname}.tar.gz {code_tarname}/


In [ ]:
# copy the deployment configs tar to a path (different from hf model artifacts)
s3_code_artifact = sess.upload_data(f"{code_tarname}.tar.gz", default_bucket, sagemaker.utils.name_from_base("htmp0615/v3"))

inference_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.8.3-cu118"

endpoint_name = sagemaker.utils.name_from_base(code_tarname.replace('_','-'))

In [ ]:
from sagemaker.model import Model
model = Model(image_uri=inference_image_uri,
              model_data=s3_code_artifact, 
              role=role)

model.deploy(initial_instance_count = 1,
             instance_type = 'ml.g4dn.2xlarge',
             endpoint_name = endpoint_name,
             container_startup_health_check_timeout = 900
            )

In [ ]:
from sagemaker import serializers, deserializers

predictor = sagemaker.Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sess,
            serializer=serializers.JSONSerializer(),
            deserializer=deserializers.JSONDeserializer(),
            )

In [ ]:
sentences_a = [['Represent the Science sentence: ','Parton energy loss in QCD matter'], 
               ['Represent the Financial statement: ','The Federal Reserve on Wednesday raised its benchmark interest rate.']]
sentences_b = [['Represent the Science sentence: ','The Chiral Phase Transition in Dissipative Dynamics'],
               ['Represent the Financial statement: ','The funds rose less than 0.5 per cent on Friday']]

In [ ]:
predictor.predict(
    {"instructs": [i[0] for i in sentences_a], 'docs': [d[1] for d in sentences_a]}
)

In [ ]:
%%timeit -n3 -r1
predictor.predict(
    {"instructs": [i[0] for i in sentences_a], 'docs': [d[1] for d in sentences_a]}
)